In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir("/content/drive/MyDrive/암빅데이터_경진대회/최종코드")

In [6]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from preprocessing.stage import Preprocess, Classify
import warnings
warnings.filterwarnings(action='ignore')

데이터 로드

In [5]:
colorectal_train = pd.read_csv("./data/original/학습데이터_대장암.csv")
colorectal_test = pd.read_csv("./data/original/대장암 합성데이터.csv")
colorectal_total = pd.concat([colorectal_train,colorectal_test], ignore_index=True)

# 학습이 잘 되도록 데이터 전체 shuffle 하기
train_shuffle = colorectal_train.sample(frac=1, random_state=23).reset_index(drop=True)  # shape: (10000,54)
test_shuffle = colorectal_test.sample(frac=1, random_state=23).reset_index(drop=True)    # shape: (5000,54)
total_shuffle = colorectal_total.sample(frac=1, random_state=23).reset_index(drop=True)

In [7]:
def Scale(df_x):
    # Type of Drink 컬럼의 값 : 1,2,3,99
    # 99(기타) -> 0으로 변경
    change_value_dict = {99:0}   # key: 변경 전 값, value: 변경 후 값
    df_x = df_x.replace({'Type of Drink' : change_value_dict})

    # df_x의 컬럼 'Type of Drink', 'Smoke'에 대하여
    # MinMaxScaler 사용하여 0~1 사이로 정규화
    scaler = MinMaxScaler()
    arr_drk = np.array(df_x['Type of Drink']).reshape(-1,1)
    scaler.fit(arr_drk)
    scaled_drk = scaler.transform(arr_drk)

    scaler = MinMaxScaler()
    arr_smk = np.array(df_x['Smoke']).reshape(-1,1)
    scaler.fit(arr_smk)
    scaled_smk = scaler.transform(arr_smk)

    # df_x로 다시 합쳐주기
    df_drk = pd.DataFrame(scaled_drk.reshape(-1,),columns=['Type of Drink'])
    df_smk = pd.DataFrame(scaled_smk.reshape(-1,),columns=['Smoke'])
    df_x = pd.concat([df_x.iloc[:,0:7], df_drk, df_smk],axis=1)

    return df_x

In [8]:
def Save_xy(data, mode):   # mode: train - "tr", test - "ts", total - "total" 
    
    ## X로 사용할 컬럼(조직학적 진단 코드 + 개인건강정보) 추출 및 저장
    df_x = pd.concat([data.iloc[:,2:9], data.iloc[:,39:41]],axis = 1)

    # X값 범위가 0~1이 되도록 결측값 0으로 변경 및 정규화
    df_x = Scale(df_x)
    # 저장
    df_x.to_csv("./data/preprocessed/data_x_"+ mode +".csv", mode = "w")
    

    ## y로 사용할 컬럼 생성 - 'Stage' : 암 병기값 1~4 소수 단위
    df_TNM = data.iloc[:,9:39]
    df_TNM['Stage'] = np.NaN   # Stage 열 추가 (초기값: NaN)

    # 암 병기값 분류를 위해 TNM 병기 값 전처리 (예: 사용 안되는 컬럼값의 값들을 유사한 컬럼에 더해주기)
    df_TNM = Preprocess(df_TNM)

    # 암 병기값 분류
    df_TNM = Classify(df_TNM)

    # df_TNM['Stage']의 NaN값을 0으로 대체
    df_TNM['Stage'].fillna(0, inplace=True)

    df_y = df_TNM['Stage']
    # 저장
    df_y.to_csv("./data/preprocessed/data_y_"+ mode +".csv", mode = "w")

    return df_x, df_y

In [9]:
df_x_tr, df_y_tr = Save_xy(train_shuffle,"tr")
df_x_ts, df_y_ts = Save_xy(test_shuffle,"ts")
df_x_total, df_y_total = Save_xy(total_shuffle,"total")